# Assessment needs mapping

In [ ]:
# Imports
import os
import numpy as np
import pandas as pd
import regex
from zipfile import ZipFile
import geopandas as gpd
from shapely.geometry import Point
import re
import calendar
from dateutil.relativedelta import relativedelta
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import contextily as ctx

## >> Input required

##### Define local authority name and filepaths

In [ ]:
local_authority_name = '...' # Local authority name to facilitate mapping
log_input_file = '.../log.csv' # Filepath to log
postcode_input_file = '.../assessment_postcode_data.xlsx' #Filepath to file containing postcodes relating to assessments
output_folder = '...' # Filepath to folder where the analysis and charts will be saved
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
risk_factor_output_folder = os.path.join(output_folder, 'Specific Risk Factors')
if not os.path.exists(risk_factor_output_folder):
    os.makedirs(risk_factor_output_folder)

## 1. Set up

##### 1.1 Define Useful References

In [ ]:
# Define risk codes and their signification
risk_codes = ['1A', '1B', '1C', '2A', '2B', '2C', '3A', '3B', '3C', '4A', '4B', '4C', '5A', '5B', '5C', '6A', '6B', '6C', '7A', 
              '8A','8B', '8C', '8D', '8E', '8F', '9A', '10A', '11A', '12A', '13A', '14A', '15A', '16A', '17A', '18A', '19A', 
              '20', '21', '22A', '23A']

risk_map = {
    '1A':'Alcohol misuse - child', 
    '1B':'Alcohol misuse - parents/carer', 
    '1C':'Alcohol misuse - other', 
    '2A':'Drug misuse - child', 
    '2B':'Drug misuse - parent/carer', 
    '2C':'Drug misuse - other', 
    '3A':'Domestic violence - child', 
    '3B':'Domestic violence - parent/carer', 
    '3C':'Domestic violence - other', 
    '4A':'Mental health - child', 
    '4B':'Mental health - parent/carer', 
    '4C':'Mental health - other', 
    '5A':'Learning disability - child', 
    '5B':'Learning disability - parent/carer', 
    '5C':'Learning disability - other', 
    '6A':'Physical disability/illness - child', 
    '6B':'Physical disability/illness - parent/carer', 
    '6C':'Physical disability/illness - other', 
    '7A':'Young carer', 
    '8A':'Privately fostered (not used anymore)',
    '8B':'Privately fostered - overseas children who intend to return', 
    '8C':'Privately fostered - overseas children who intend to stay', 
    '8D':'Privately fostered - UK children in educational placements', 
    '8E':'Privately fostered - UK children making alternative family arrangements', 
    '8F':'Privately fostered - other', 
    '9A':'Unaccompanied Asylum Seeking Child', 
    '10A':'Missing', 
    '11A':'Child sexual exploitation', 
    '12A':'Trafficking', 
    '13A':'Gangs', 
    '14A':'Socially unacceptable behaviour', 
    '15A':'Self-harm', 
    '16A':'Neglect', 
    '17A':'Emotional abuse', 
    '18A':'Physical abuse', 
    '19A':'Sexual abuse', 
    '20':'Other', 
    '21':'No factors identified', 
    '22A':'Female genital mutilation', 
    '23A':'Abuse linked to faith or belief'
}

code_map = {
    '1':'Alcohol Abuse', 
    '2':'Drug Abuse', 
    '3':'Domestic Violence', 
    '4':'Mental Health', 
    '5':'Learning Disability', 
    '6':'Physical Disability', 
    '7A':'Young Carer', 
    '8':'Privately fostered', 
    '9A':'UASC', 
    '10A':'Missing', 
    '11A':'Sexual exploitation', 
    '12A':'Trafficking', 
    '13A':'Gangs', 
    '14A':'Behaviour', 
    '15A':'Self-harm', 
    '16A':'Neglect', 
    '17A':'Emotional abuse', 
    '18A':'Physical abuse', 
    '19A':'Sexual abuse', 
    '20':'Other', 
    '21':'No factors id.', 
    '22A':'FGM', 
    '23A':'Abuse Linked to Faith/belief'
}

##### 1.2 Read in geography related data

In [ ]:
# Download all of the geography files (these can be found in the readme) and specify the path to their location

# filepath to postcode geographic information
postcode_geography_file = '.../ukpostcodes.zip' #Postcode file location
lsoa_shapes_file = '.../geo-lsoa.zip' #LSOA shapes file location
lau_shapes_file = './supporting_data/geo-lau.geojson.bz2' #LAU shapes file location

In [ ]:
# Read in postcode_geography_file to postcode_lat_lon
with ZipFile(postcode_geography_file) as myzip:
    with myzip.open('ukpostcodes.csv') as myfile:
        postcode_lat_lon = pd.read_csv(myfile)

postcode_lat_lon = postcode_lat_lon[['postcode','latitude','longitude']] 
postcode_lat_lon.head(1)

In [ ]:
# Read in lsoa_shapes_file to lsoas
lsoas = gpd.read_file(lsoa_shapes_file)
lsoas = lsoas[["lsoa11cd", "lsoa11nm", "geometry"]]
lsoas.head(1)

In [ ]:
# Read in lau_shapes_file (local authority) to laus
laus = gpd.read_file(lau_shapes_file)
laus = laus[["lau118cd","lau118nm","geometry"]]
laus.head(1)

## 2 Define Functions

##### 2.1 Define General Functions

In [ ]:
# Def start_end_date function giving start date and end date for the analysis

def start_end_date(date, period):
    '''
    Returns start and end date based on a given date and given period (months)
    The given date must be a Timestamp
    The end date will be the last day of the month prior to the given date
    The start date will be the first day of the month after doing (end date - period)
    '''
    # If given date is already at the end of the month
    if date.day == calendar.monthrange(date.year, date.month)[1]:
        end_date = date
        start_date = end_date + relativedelta(months=-(period-1))
        start_date = start_date.replace(day=1)
    # If given date is not at the end of the month
    else:
        end_date_intermediate = date.replace(day=1)
        end_date = end_date_intermediate - np.timedelta64(1, 'D')
        start_date = end_date_intermediate + relativedelta(months=-period)
    try:
        start_date = start_date.replace(hour = 0, minute = 0, second = 0)
    except:
        print('No time in start date')
    try:
        end_date = end_date.replace(hour = 23, minute = 59, second = 59)
    except:
        print('No time in end date')
    # Turn into pd-compatible format
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)
    return start_date, end_date

## 3 Analysis

##### 3.1 Create base dataframe: risk_factors_and_postcodes

In [ ]:
# Load log input file
log = pd.read_csv(log_input_file)
# Convert date column to datetime
log['Date'] = pd.to_datetime(log['Date'])

# Load local authority postcode file
postcodes = pd.read_excel(postcode_input_file)
# Ensure column names are correct for future matching
postcodes.columns = ['CUID', 'Date','Postcode']
# Convert date column to datetime
postcodes['Date'] = pd.to_datetime(postcodes['Date'])

# Filter log to contain only CIN Census Assessment Close datapoints
log_assessment_close = log[log.Type == 'AssessmentAuthorisationDate']
# Define columns that we want to keep in the new dataframe
relevant_columns = ['CUID', 'Date','Type']
# Create risk factor log, by splitting the Factors column and counting the values into new columns
risk_factor_log = pd.concat([log_assessment_close[relevant_columns],log_assessment_close['Factors'].str.split(',', expand = True).stack().str.get_dummies().sum(level=0)],1)

# Merge risk factors with postcodes
risk_factors_and_postcodes = risk_factor_log.merge(postcodes, how = 'left', on =['CUID','Date'])
risk_factors_and_postcodes.head()

##### 3.2 Add area data to base dataframe: risk_factors_with_areas

In [ ]:
# Remove all entries without postcodes
risk_factors_with_areas = risk_factors_and_postcodes[~risk_factors_and_postcodes.Postcode.isnull()]
# Merge postcode_lat_lon onto risk_factors_with_areas to add the latitude and longitude data
risk_factors_with_areas = risk_factors_with_areas.merge(postcode_lat_lon, on = 'Postcode', how = 'left')
# Drop any rows where 'Latitude' or 'Longitude' are not populated (some newer postcodes are not in postcodes_lat_lon database)
risk_factors_with_areas = risk_factors_with_areas.dropna(subset=["Latitude", "Longitude"])
# Rename columns to facilitate creation of coordinates
risk_factors_with_areas = risk_factors_with_areas.rename(columns = {'Longitude':'longitude','Latitude':'latitude'})

# Create a series of coordinates of coordinates, by applying 'Point' to latitude and longitude columns 
coords = pd.Series(list(zip(risk_factors_with_areas.longitude,risk_factors_with_areas.latitude))).apply(Point)
crs = {'init' :'epsg:4326'} # Set geometry to match our maps
# Turn coordinates into a GeoDataFrame, which will be used to determine the LAUs and LSOAs the postcodes sit within
coords = gpd.GeoDataFrame(pd.DataFrame(coords, 
            columns=["coordinates"]),crs = crs, geometry="coordinates")

# Add new lsoa columns to risk_factors_with_areas by joining lsoas dataframe to new coordinates DataFrame
risk_factors_with_areas[["lsoa11cd","lsoa11nm"]] = gpd.sjoin(coords, lsoas, how="left", 
                                                             op="intersects")[["lsoa11cd","lsoa11nm"]]

# Add new lau columns to risk_factors_with_areas by joining laus dataframe to new coordinates DataFrame
risk_factors_with_areas[["lau118cd","lau118nm"]] = gpd.sjoin(coords, laus, how="left", op="intersects")[["lau118cd","lau118nm"]]

##### 3.3 Regroup risk factors to create more general categories and drop entries from outside LA: risk_factors_with_areas_regrouped

In [ ]:
# Define regroup function for regrouping similar factors to create more general categories
def regroup(df, factors_to_group):
    columns = df.columns.tolist()
    for factor in factors_to_group:
        regex = r",({}[A-Z])".format(factor)
        group = re.findall(regex, ','.join(columns))
        df[factor] = df[group].sum(axis=1)
        df.drop(labels=group, axis=1, inplace=True)
    return df

risk_factors_with_areas_regrouped = risk_factors_with_areas.copy()

# Define factors that should be regrouped
factors_for_regrouping = ['1', '2', '3', '4', '5', '6', '8']
risk_factors_with_areas_regrouped = regroup(risk_factors_with_areas_regrouped, factors_for_regrouping)

# Set the index on Date
risk_factors_with_areas_regrouped = risk_factors_with_areas_regrouped.set_index('Date')

# Drop entries from outside the local authority to facilitate mapping
risk_factors_with_areas_regrouped = risk_factors_with_areas_regrouped[risk_factors_with_areas_regrouped.lau118nm == local_authority_name]

##### 3.4 Create DataFrame of overall volume counts by Risk Factor: risk_factors_current_previous

In [ ]:
# Define a function to create an annual count of risk factors across a local authority
# Note: column_id needs to be specified to be able to differentiate the different DataFrames in the next step
def calculate_annual_risk_factor_volumes(df, year_end_date, column_id):
    start_date, end_date = start_end_date(year_end_date, 12)
    filtered_df = df[(df.index >= start_date)&(df.index <= end_date)]
    annual_count_of_risk_factors = pd.DataFrame(filtered_df[list_of_risk_columns].sum(), columns = [column_id])
    return filtered_df, annual_count_of_risk_factors

# Create list of risk columns for formatting
list_of_risk_columns = ['1', '2', '3', '4', '5', '6','7A', '8', '9A','10A', '11A', '12A', '13A', '14A', '15A', '16A',
       '17A', '18A', '19A', '20', '21', '22A', '23A']

In [ ]:
# Create DataFrame of current and previous year risk factors, with comparison

# Define current year end date
latest_year_end_date = risk_factors_with_areas_regrouped.index.max()
# Define previous year end date
prev_year_end_date = risk_factors_with_areas_regrouped.index.max() - relativedelta(years=1)

# Create current year count of risk factors
risk_factors_current, la_risk_count_current = calculate_annual_risk_factor_volumes(risk_factors_with_areas_regrouped, latest_year_end_date, 'Current Year')
# Create previous year count of risk factors
risk_factors_prev, la_risk_count_prev = calculate_annual_risk_factor_volumes(risk_factors_with_areas_regrouped, prev_year_end_date, 'Previous Year')

# Merge the current and previous year DataFrames to create comparison DataFrame
risk_factors_current_previous = la_risk_count_current.merge(la_risk_count_prev, how = 'left',left_index=True, right_index = True)

# Caclulate a column showing change from previous year
risk_factors_current_previous['% Change from Previous Year'] = (risk_factors_current_previous['Current Year']-risk_factors_current_previous['Previous Year'])/(risk_factors_current_previous['Previous Year'])

# Map the risk factor codes to their names for easier understanding
risk_factors_current_previous['Risk Factor'] = risk_factors_current_previous.index.map(code_map)
risk_factors_current_previous.to_excel(os.path.join(output_folder,'risk_factors_current_previous.xlsx'))

##### 3.5 Identify risk factors to map (either by highest volume or highest change)

In [ ]:
# Calculate top 5 risk factors by volume
top_5_risk_factors_volume = list(risk_factors_current_previous.nlargest(5, columns = 'Current Year').index)
# Filter dataframe to find top 5 risk factors to look at
top_5_risk_factors_growth = list(risk_factors_current_previous.nlargest(5, columns = '% Change from Previous Year').index)

## 4 Mapping

### 4.1 Mapping Overall Assessments

##### 4.1.1. Define Functions

In [ ]:
# Define the create total assessments df function, including volumes and changes in assessment by lsoa, and the lsoa geometry
def create_total_assessments_df(df_current,df_previous):
    # Group current year data by lsoa
    current_year = pd.DataFrame(df_current.groupby('lsoa11nm')['CUID'].count())
    # Group previous year data by lsoa
    previous_year = pd.DataFrame(df_previous.groupby('lsoa11nm')['CUID'].count())
    # Merge the two together
    current_and_prev_year = current_year.merge(previous_year, how='left', left_index=True, right_index = True).reset_index()
    current_and_prev_year.columns = ['lsoa11nm', 'Total Current Year', 'Total Previous Year']
    # Calculate change from previous year (in absolute values, rather than % change)
    current_and_prev_year['Change from Previous Year'] = current_and_prev_year['Total Current Year'] - current_and_prev_year['Total Previous Year']
    # Add Geometry
    total_assessments_df = lsoas.merge(current_and_prev_year, how='right', on = 'lsoa11nm')
    # Save file in case of desire to review
    total_assessments_df.to_excel(os.path.join(output_folder, 'Assessment Prevalence and Change by LSOA.xlsx'))
    return total_assessments_df

# Define create_assessment_prevalence function, which plots the volume of assessments by lsoa, and saves to output folder
def create_assessment_prevalence(df, authority):
    # Filter for only the shape of the relevant local authority
    la_shape = laus[laus['lau118nm']==authority]
    # Ensure coordinate systems are the same
    df = df.to_crs(epsg=3857)
    la_shape = la_shape.to_crs(epsg=3857)
    # Specify map parameters
    fig, ax = plt.subplots(1, figsize=(9,9))
    plt.tight_layout()
    plt.title("{} - Total Assessments".format(authority), size = 15)
    plt.axis('off')
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.1)
    plot_cfg = {"vmin": 0, # This is 0, as there will be no 'negative' entries
                "linewidth": 0.05, 
                "cmap": 'OrRd',
                "legend": True,
                "alpha": .8}
    la_shape.plot(ax=ax, 
                  color='none', 
                  edgecolor='black', 
                  linewidth=5)
    df.plot(ax=ax, column='Total Current Year', **plot_cfg, 
                  legend_kwds={'label':'Volume of Assessments'}, cax=cax)
    # Add map beneath graph
    try:
        ctx.add_basemap(ax)
    except:
        print("Fetching map resulted in error for {}".format(authority))
        pass
    # Save figure to output folders
    plt.savefig(os.path.join(output_folder, '{} - Total Assessments.png'.format(authority)),bbox_inches = "tight")
    
# Create Map of Prevalence
def create_change_in_assessment_prevalence(df, authority):
    # Filter for only the shape of the relevant local authority
    la_shape = laus[laus['lau118nm']==authority]
    # Ensure coordinate systems are the same
    df = df.to_crs(epsg=3857)
    la_shape = la_shape.to_crs(epsg=3857)
    # Specify maximum change value to ensure legend is equal on both sides (feeds into vmin and vmax parameters)
    maximum_change_value = max(df['Change from Previous Year'].max(), -df['Change from Previous Year'].min())
    # Specify graph parameters
    fig, ax = plt.subplots(1, figsize=(9,9))
    plt.tight_layout()
    plt.title("{} - Change in Assessments".format(authority),size = 15)
    plt.axis('off')
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.1)
    plot_cfg = {
                "vmin": -maximum_change_value,
                "vmax": maximum_change_value,
                "linewidth": 0.05, 
                "cmap": 'RdYlGn_r',
                "legend": True,
                "alpha": .8}
    la_shape.plot(ax=ax, color='none', edgecolor='black', linewidth=5)
    df.plot(ax=ax, column='Change from Previous Year', **plot_cfg,
            legend_kwds={'label':'Change in Volume of Assessments'}, cax = cax)
    # Add map beneath graph
    try:
        ctx.add_basemap(ax)
    except:
        print("Fetching map resulted in error for {}".format(authority))
        pass
    # Save figure to output folder
    plt.savefig(os.path.join(output_folder, '{} - Change in Assessments.png'.format(authority)),bbox_inches = "tight")

##### 4.1.2 Create Maps

In [ ]:
# Create total assessments DataFrame using risk_factors_current and risk_factors_prev from section 3.4
total_assessments_df = create_total_assessments_df(risk_factors_current, risk_factors_prev)

# Create assessment prevalence and change in assessment prevalence graphs, based on total_assessments_df
create_assessment_prevalence(total_assessments_df, local_authority_name)
create_change_in_assessment_prevalence(total_assessments_df, local_authority_name)

### 4.2 Mapping Specific Risk Factors

##### 4.2.1. Define Functions

In [ ]:
# Define function to create DataFrame driving risk factor specific maps, showing volume, change and geometry
def create_risk_factor_lsoa_df(risk_factor):
    # Group current and prev year data by lsoa for a specified risk factor
    current_year = pd.DataFrame(risk_factors_current.groupby('lsoa11nm')[risk_factor].sum())
    previous_year = pd.DataFrame(risk_factors_prev.groupby('lsoa11nm')[risk_factor].sum())
    # Merge the two together
    current_and_prev_year = current_year.merge(previous_year, how='left', left_index=True, right_index = True).reset_index()
    current_and_prev_year.columns = ['lsoa11nm', 'Total Current Year', 'Total Previous Year']
    # Caclulate change from previous year
    current_and_prev_year['Change from Previous Year'] = current_and_prev_year['Total Current Year'] - current_and_prev_year['Total Previous Year']
    # Add Geometry
    lsoa_risk_factors_and_geometry = lsoas.merge(current_and_prev_year, how='right', on = 'lsoa11nm')
    return lsoa_risk_factors_and_geometry

# Define function to create prevalence graphs by risk factors
def create_prevalence_graphs(risk_factor, local_authority):
    # Define LA Shape
    la_shape = laus[laus['lau118nm']==local_authority]
    # Create DataFrame for specified risk factor
    df = create_risk_factor_lsoa_df(risk_factor)
    # Access risk factor name for titles
    risk_factor_name = code_map[risk_factor]
    # Save file in case of desire to review
    df.to_excel(os.path.join(risk_factor_output_folder, '{} Risk Factor Prevalence and Change by LSOA.xlsx'.format(risk_factor_name)))
    # Ensure coordinate systems are the same
    df = df.to_crs(epsg=3857)
    la_shape = la_shape.to_crs(epsg=3857)
    # Specify graph parameters
    fig, ax = plt.subplots(1, figsize=(9,9))
    plt.tight_layout()
    plt.title("{} - Assessments Showing Risk Factor {} ".format(local_authority, risk_factor_name), size = 15)
    plt.axis('off')
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.1)
    plot_cfg = {
                "vmin": 0,
                "linewidth": 0.05, 
                "cmap": 'OrRd',
                "legend": True,
                "alpha": .8}
    la_shape.plot(ax=ax, color='none', edgecolor='black', linewidth=5)
    df.plot(ax=ax, column='Total Current Year', **plot_cfg,legend_kwds={'label':'Volume of Assessments with Need Identified'}, cax = cax)
    # Add map underneath
    try:
        ctx.add_basemap(ax)
    except:
        print("Fetching map resulted in error for {}".format(local_authority))
        pass
    # Save figure to output folder
    plt.savefig(os.path.join(risk_factor_output_folder, '{} - Prevalence of Risk Factor {}.png'.format(local_authority, risk_factor_name)),bbox_inches = "tight")
    
def create_change_graphs(risk_factor, local_authority):
    # Define LA Shape
    la_shape = laus[laus['lau118nm']==local_authority]
    # Create DataFrame for specified risk factor
    df = create_risk_factor_lsoa_df(risk_factor)
    # Access risk factor name for titles
    risk_factor_name = code_map[risk_factor]
    # Remove any values where 'Change from Previous Year' is null, they will break the graph otherwise
    df = df[df['Change from Previous Year'].isnull() == False]
    # Calculate maximum change value to input into legend
    maximum_change_value = max(df['Change from Previous Year'].max(), -df['Change from Previous Year'].min())
    # Ensure coordinate systems are the same
    df = df.to_crs(epsg=3857)
    la_shape = la_shape.to_crs(epsg=3857)
    # Specify graph paramaters
    fig, ax = plt.subplots(1, figsize=(9,9))
    plt.title("{} - Change from Previous Year Risk Factor {} ".format(local_authority, risk_factor_name), size =15)
    plt.axis('off')
    plt.tight_layout()
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.1)
    plot_cfg = {
                "vmin": -maximum_change_value,
                "vmax": maximum_change_value,
                "linewidth": 0.05, 
                "cmap": 'RdYlGn_r',
                "legend": True,
                "alpha": .8}
    la_shape.plot(ax=ax, color='none', edgecolor='black', linewidth=5)
    df.plot(ax=ax, column='Change from Previous Year', **plot_cfg,legend_kwds={'label':'Change in Volume of Assessments with Need Identified'}, cax = cax)
    # Add underlying map
    try:
        ctx.add_basemap(ax)
    except:
        print("Fetching map resulted in error for {}".format(local_authority))
        pass
    # Save figure to risk factors output folder
    plt.savefig(os.path.join(risk_factor_output_folder, '{} - Change in Risk Factor {}.png'.format(local_authority, risk_factor_name)),bbox_inches = "tight")

In [ ]:
# Create risk factors specific maps for top 5 risk factors by volume
for risk_factors in top_5_risk_factors_volume:
    create_prevalence_graphs(risk_factors, local_authority_name)
    create_change_graphs(risk_factors, local_authority_name)